# Province Gender Available - Sample Thousand Users

In [1]:
import gzip
import json
import pandas as pd
import tweepy

## Read Data from Gzip

In [2]:
with gzip.open("province_gender_available_metadata_added-220626_combined.txt.gz", "rb") as f:
    users = f.readlines()

In [3]:
for i in range(len(users)):
    users[i] = json.loads(users[i].decode("utf-8"))

In [4]:
len(users)

728465

## Create DataFrame

In [5]:
df = pd.DataFrame(users)

In [6]:
df.shape

(728465, 15)

In [7]:
df.head(3)

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded
0,,size ait oldunu sandıgınız şey nedense siz öld...,sahin kartaltepe,kartaltepesahin,1317807690010316800,Sun Oct 18 12:40:37 +0000 2020,"[{'source': 'description', 'pcode': 17}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1233032192508465153, 218819376, 1156665621242...",[],[],0,4,1317809402875961345/tsQsT3ET_normal.jpg,220614
1,"ankara, türkiye",박 혜 지 #kühf,karahindiba 🌱,betulkacak,337741849,Mon Jul 18 14:23:00 +0000 2011,"[{'source': 'location', 'pcode': 6}]","[{'source': 'screen_name', 'gender': 'female'}]",[],[],[],95,66,1049265032943607809/Z1yokA9T_normal.jpg,220614
2,"mersin, türkiye",play fair in love...,özge ⚜,oztapiklama,1181665909804617728,Tue Oct 08 20:21:28 +0000 2019,"[{'source': 'location', 'pcode': 33}]","[{'source': 'name', 'gender': 'female'}]",[],[],[],34,52,1530164796800307200/CvI5L318_normal.jpg,220614


### Delete Users without Tweets

In [8]:
# Remove users without tweets in df
no_tweet_indices = []
for i in range(len(df)):
    if len(df.loc[i, "tweets"]) == 0:
        no_tweet_indices.append(i)
        
df = df.drop(no_tweet_indices).reset_index(drop=True)

In [9]:
df.shape

(549205, 15)

In [10]:
# Remove users without tweets in users
users_with_tweets = []
for user in users:
    if user["tweets"]:
        users_with_tweets.append(user)

users = users_with_tweets

### Select Active Users (last 2 years)

In [12]:
activity = []

for i in range(len(df)):
    n_years = 2
    threshold = int(str(int(df.loc[i, "downloaded"][:2]) - n_years) + df.loc[i, "downloaded"][2:])
    
    return_val = "passive"
    
    if not df.loc[i, "tweets"]:
        return_val = "passive"
    else:
        tweet_dates = [int(tweet["twt_date"]) for tweet in df.loc[i, "tweets"]]
        if any([tweet_date >= threshold for tweet_date in tweet_dates]):
            return_val = "active"
        else:
            return_val = "passive"
            
    activity.append(return_val)

In [13]:
df["activity"] = pd.Series(activity, name="activity")

In [14]:
df = df[df["activity"] == "active"].reset_index(drop=True)

In [15]:
df.shape

(248239, 16)

### Check if User is still Active

In [16]:
with open("bearer_tokens.txt", "r") as f:
    bearer_tokens = f.read().split("\n")
    
bearer_token = bearer_tokens[0]

In [17]:
## tweepy.Client
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

In [18]:
def get_users(user_ids):
    """
    - Creates user id batches consisting of 100 user ids from the user_ids list.
    - Requests "created_at", "description", "entities", "id", "location", "name",
    "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "username",
    "verified", "withheld" fields for users in each batch, returns the whole list.
    """
    users = []
    
    user_batches = [user_ids[i:i+100] for i in range(0, len(user_ids), 100)]
    
    for i, batch in enumerate(user_batches):
        print(f"{i+1}/{len(user_batches)} | {(i+1)/len(user_batches)*100:.2f}%")
        
        # Exception handling for ConnectionResetError
        response = None
        while not response:
            try:
                response = client.get_users(ids=batch, user_fields=["created_at",
                                                                    "description",
                                                                    "entities",
                                                                    "id",
                                                                    "location",
                                                                    "name",
                                                                    "pinned_tweet_id",
                                                                    "profile_image_url",
                                                                    "protected",
                                                                    "public_metrics",
                                                                    "url",
                                                                    "username",
                                                                    "verified",
                                                                    "withheld"])
                for user in response.data:
                    users.append(user.data)
            except:
                continue
            
    return pd.DataFrame(users)

In [19]:
active_users = get_users(list(df["id_str"]))

1/2483 | 0.04%
2/2483 | 0.08%
3/2483 | 0.12%
4/2483 | 0.16%
5/2483 | 0.20%
6/2483 | 0.24%
7/2483 | 0.28%
8/2483 | 0.32%
9/2483 | 0.36%
10/2483 | 0.40%
11/2483 | 0.44%
12/2483 | 0.48%
13/2483 | 0.52%
14/2483 | 0.56%
15/2483 | 0.60%
16/2483 | 0.64%
17/2483 | 0.68%
18/2483 | 0.72%
19/2483 | 0.77%
20/2483 | 0.81%
21/2483 | 0.85%
22/2483 | 0.89%
23/2483 | 0.93%
24/2483 | 0.97%
25/2483 | 1.01%
26/2483 | 1.05%
27/2483 | 1.09%
28/2483 | 1.13%
29/2483 | 1.17%
30/2483 | 1.21%
31/2483 | 1.25%
32/2483 | 1.29%
33/2483 | 1.33%
34/2483 | 1.37%
35/2483 | 1.41%
36/2483 | 1.45%
37/2483 | 1.49%
38/2483 | 1.53%
39/2483 | 1.57%
40/2483 | 1.61%
41/2483 | 1.65%
42/2483 | 1.69%
43/2483 | 1.73%
44/2483 | 1.77%
45/2483 | 1.81%
46/2483 | 1.85%
47/2483 | 1.89%
48/2483 | 1.93%
49/2483 | 1.97%
50/2483 | 2.01%
51/2483 | 2.05%
52/2483 | 2.09%
53/2483 | 2.13%
54/2483 | 2.17%
55/2483 | 2.22%
56/2483 | 2.26%
57/2483 | 2.30%
58/2483 | 2.34%
59/2483 | 2.38%
60/2483 | 2.42%
61/2483 | 2.46%
62/2483 | 2.50%
63/2483 | 2.54%
6

Rate limit exceeded. Sleeping for 788 seconds.


300/2483 | 12.08%
301/2483 | 12.12%
302/2483 | 12.16%
303/2483 | 12.20%
304/2483 | 12.24%
305/2483 | 12.28%
306/2483 | 12.32%
307/2483 | 12.36%
308/2483 | 12.40%
309/2483 | 12.44%
310/2483 | 12.48%
311/2483 | 12.53%
312/2483 | 12.57%
313/2483 | 12.61%
314/2483 | 12.65%
315/2483 | 12.69%
316/2483 | 12.73%
317/2483 | 12.77%
318/2483 | 12.81%
319/2483 | 12.85%
320/2483 | 12.89%
321/2483 | 12.93%
322/2483 | 12.97%
323/2483 | 13.01%
324/2483 | 13.05%
325/2483 | 13.09%
326/2483 | 13.13%
327/2483 | 13.17%
328/2483 | 13.21%
329/2483 | 13.25%
330/2483 | 13.29%
331/2483 | 13.33%
332/2483 | 13.37%
333/2483 | 13.41%
334/2483 | 13.45%
335/2483 | 13.49%
336/2483 | 13.53%
337/2483 | 13.57%
338/2483 | 13.61%
339/2483 | 13.65%
340/2483 | 13.69%
341/2483 | 13.73%
342/2483 | 13.77%
343/2483 | 13.81%
344/2483 | 13.85%
345/2483 | 13.89%
346/2483 | 13.93%
347/2483 | 13.98%
348/2483 | 14.02%
349/2483 | 14.06%
350/2483 | 14.10%
351/2483 | 14.14%
352/2483 | 14.18%
353/2483 | 14.22%
354/2483 | 14.26%
355/2483 |

Rate limit exceeded. Sleeping for 803 seconds.


600/2483 | 24.16%
601/2483 | 24.20%
602/2483 | 24.24%
603/2483 | 24.29%
604/2483 | 24.33%
605/2483 | 24.37%
606/2483 | 24.41%
607/2483 | 24.45%
608/2483 | 24.49%
609/2483 | 24.53%
610/2483 | 24.57%
611/2483 | 24.61%
612/2483 | 24.65%
613/2483 | 24.69%
614/2483 | 24.73%
615/2483 | 24.77%
616/2483 | 24.81%
617/2483 | 24.85%
618/2483 | 24.89%
619/2483 | 24.93%
620/2483 | 24.97%
621/2483 | 25.01%
622/2483 | 25.05%
623/2483 | 25.09%
624/2483 | 25.13%
625/2483 | 25.17%
626/2483 | 25.21%
627/2483 | 25.25%
628/2483 | 25.29%
629/2483 | 25.33%
630/2483 | 25.37%
631/2483 | 25.41%
632/2483 | 25.45%
633/2483 | 25.49%
634/2483 | 25.53%
635/2483 | 25.57%
636/2483 | 25.61%
637/2483 | 25.65%
638/2483 | 25.69%
639/2483 | 25.73%
640/2483 | 25.78%
641/2483 | 25.82%
642/2483 | 25.86%
643/2483 | 25.90%
644/2483 | 25.94%
645/2483 | 25.98%
646/2483 | 26.02%
647/2483 | 26.06%
648/2483 | 26.10%
649/2483 | 26.14%
650/2483 | 26.18%
651/2483 | 26.22%
652/2483 | 26.26%
653/2483 | 26.30%
654/2483 | 26.34%
655/2483 |

...

1800/2483 | 72.49%
1801/2483 | 72.53%
1802/2483 | 72.57%
1803/2483 | 72.61%
1804/2483 | 72.65%
1805/2483 | 72.69%
1806/2483 | 72.73%
1807/2483 | 72.77%
1808/2483 | 72.82%
1809/2483 | 72.86%
1810/2483 | 72.90%
1811/2483 | 72.94%
1812/2483 | 72.98%
1813/2483 | 73.02%
1814/2483 | 73.06%
1815/2483 | 73.10%
1816/2483 | 73.14%
1817/2483 | 73.18%
1818/2483 | 73.22%
1819/2483 | 73.26%
1820/2483 | 73.30%
1821/2483 | 73.34%
1822/2483 | 73.38%
1823/2483 | 73.42%
1824/2483 | 73.46%
1825/2483 | 73.50%
1826/2483 | 73.54%
1827/2483 | 73.58%
1828/2483 | 73.62%
1829/2483 | 73.66%
1830/2483 | 73.70%
1831/2483 | 73.74%
1832/2483 | 73.78%
1833/2483 | 73.82%
1834/2483 | 73.86%
1835/2483 | 73.90%
1836/2483 | 73.94%
1837/2483 | 73.98%
1838/2483 | 74.02%
1839/2483 | 74.06%
1840/2483 | 74.10%
1841/2483 | 74.14%
1842/2483 | 74.18%
1843/2483 | 74.22%
1844/2483 | 74.27%
1845/2483 | 74.31%
1846/2483 | 74.35%
1847/2483 | 74.39%
1848/2483 | 74.43%
1849/2483 | 74.47%
1850/2483 | 74.51%
1851/2483 | 74.55%
1852/2483 | 

Rate limit exceeded. Sleeping for 800 seconds.


2100/2483 | 84.58%
2101/2483 | 84.62%
2102/2483 | 84.66%
2103/2483 | 84.70%
2104/2483 | 84.74%
2105/2483 | 84.78%
2106/2483 | 84.82%
2107/2483 | 84.86%
2108/2483 | 84.90%
2109/2483 | 84.94%
2110/2483 | 84.98%
2111/2483 | 85.02%
2112/2483 | 85.06%
2113/2483 | 85.10%
2114/2483 | 85.14%
2115/2483 | 85.18%
2116/2483 | 85.22%
2117/2483 | 85.26%
2118/2483 | 85.30%
2119/2483 | 85.34%
2120/2483 | 85.38%
2121/2483 | 85.42%
2122/2483 | 85.46%
2123/2483 | 85.50%
2124/2483 | 85.54%
2125/2483 | 85.58%
2126/2483 | 85.62%
2127/2483 | 85.66%
2128/2483 | 85.70%
2129/2483 | 85.74%
2130/2483 | 85.78%
2131/2483 | 85.82%
2132/2483 | 85.86%
2133/2483 | 85.90%
2134/2483 | 85.94%
2135/2483 | 85.98%
2136/2483 | 86.02%
2137/2483 | 86.07%
2138/2483 | 86.11%
2139/2483 | 86.15%
2140/2483 | 86.19%
2141/2483 | 86.23%
2142/2483 | 86.27%
2143/2483 | 86.31%
2144/2483 | 86.35%
2145/2483 | 86.39%
2146/2483 | 86.43%
2147/2483 | 86.47%
2148/2483 | 86.51%
2149/2483 | 86.55%
2150/2483 | 86.59%
2151/2483 | 86.63%
2152/2483 | 

Rate limit exceeded. Sleeping for 799 seconds.


2400/2483 | 96.66%
2401/2483 | 96.70%
2402/2483 | 96.74%
2403/2483 | 96.78%
2404/2483 | 96.82%
2405/2483 | 96.86%
2406/2483 | 96.90%
2407/2483 | 96.94%
2408/2483 | 96.98%
2409/2483 | 97.02%
2410/2483 | 97.06%
2411/2483 | 97.10%
2412/2483 | 97.14%
2413/2483 | 97.18%
2414/2483 | 97.22%
2415/2483 | 97.26%
2416/2483 | 97.30%
2417/2483 | 97.34%
2418/2483 | 97.38%
2419/2483 | 97.42%
2420/2483 | 97.46%
2421/2483 | 97.50%
2422/2483 | 97.54%
2423/2483 | 97.58%
2424/2483 | 97.62%
2425/2483 | 97.66%
2426/2483 | 97.70%
2427/2483 | 97.74%
2428/2483 | 97.78%
2429/2483 | 97.83%
2430/2483 | 97.87%
2431/2483 | 97.91%
2432/2483 | 97.95%
2433/2483 | 97.99%
2434/2483 | 98.03%
2435/2483 | 98.07%
2436/2483 | 98.11%
2437/2483 | 98.15%
2438/2483 | 98.19%
2439/2483 | 98.23%
2440/2483 | 98.27%
2441/2483 | 98.31%
2442/2483 | 98.35%
2443/2483 | 98.39%
2444/2483 | 98.43%
2445/2483 | 98.47%
2446/2483 | 98.51%
2447/2483 | 98.55%
2448/2483 | 98.59%
2449/2483 | 98.63%
2450/2483 | 98.67%
2451/2483 | 98.71%
2452/2483 | 

In [20]:
active_users.head()

,verified,pinned_tweet_id,id,profile_image_url,entities,description,created_at,name,public_metrics,protected,username,url,location,withheld
0,False,1427014278712893444,1149269630088896514,https://pbs.twimg.com/profile_images/151019963...,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",,2019-07-11T10:50:06.000Z,tolstoyunbisikleti🕊️,"{'followers_count': 326, 'following_count': 30...",False,karahasanil,https://t.co/Y3JzF1dxKw,Ordu,NaN
1,False,NaN,3632138895,https://pbs.twimg.com/profile_images/133607292...,NaN,o gemi bir gun gelecek be ismail abi \ngalatas...,2015-09-12T14:46:25.000Z,Ali Uzun,"{'followers_count': 67, 'following_count': 94,...",False,AliUzun60320105,,"Alibeyköy Merkez, İstanbul",NaN
2,False,NaN,932820426933264384,https://pbs.twimg.com/profile_images/133882654...,NaN,Galatasaray sevdalısı,2017-11-21T03:58:15.000Z,Akif Çelik,"{'followers_count': 24, 'following_count': 78,...",False,akifcelik405,,"Ankara, Türkiye",NaN
3,False,NaN,1283126077196468224,https://pbs.twimg.com/profile_images/128385823...,NaN,Mutlu,2020-07-14T19:48:07.000Z,Mehmet,"{'followers_count': 0, 'following_count': 6, '...",False,Mehmet57455717,,NaN,NaN
4,False,1468888530923499520,1174227612681625601,https://pbs.twimg.com/profile_images/151429722...,NaN,FENERBAHÇE\nZiraaat Mühendisi 👷🏼‍♀️\nİSTANBUL,2019-09-18T07:44:35.000Z,CEREN UZUNEL,"{'followers_count': 585, 'following_count': 87...",False,Cerenimoou,,NaN,NaN


In [21]:
df = df[df["id_str"].isin(active_users["id"])].reset_index(drop=True)

In [22]:
df.shape

(246855, 16)

### Profile Image URL

In [23]:
df["pp"] = df["pp"].apply(lambda x: "https://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png" if x == "" else "https://pbs.twimg.com/profile_images/" + x.replace("normal", "400x400"))

### User URL

In [24]:
df["url"] = df["screen_name"].apply(lambda x: "https://twitter.com/" + x)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246855 entries, 0 to 246854
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   location         246855 non-null  object
 1   description      246855 non-null  object
 2   name             246855 non-null  object
 3   screen_name      246855 non-null  object
 4   id_str           246855 non-null  object
 5   created_at       246855 non-null  object
 6   province_codes   246855 non-null  object
 7   genders          246855 non-null  object
 8   following        246855 non-null  object
 9   followers        246855 non-null  object
 10  tweets           246855 non-null  object
 11  followers_count  246855 non-null  object
 12  following_count  246855 non-null  object
 13  pp               246855 non-null  object
 14  downloaded       246855 non-null  object
 15  activity         246855 non-null  object
 16  url              246855 non-null  object
dtypes: object(

In [31]:
df.to_csv("filtered_users-220702.csv", index=False)

### Sample 1000 Users

In [27]:
sample_df = df.sample(1000, random_state=0).reset_index(drop=True)

for i in range(0, 1001, 60):
    display(sample_df.iloc[i:i+60,:])

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
0,"antalya, türkiye",ofroadcı,suleyman bıcakcı,suleymanbcakci,1250059171275440130,Tue Apr 14 13:51:56 +0000 2020,"[{'source': 'location', 'pcode': 7}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1350847076570169345, 1399076882365222914, 117...","[1436345539797344285, 1483841371119173633, 148...","[{'ref_twt_id_str': '1466345118873620480', 're...",159,62,https://pbs.twimg.com/profile_images/126065747...,220323,active,https://twitter.com/suleymanbcakci
1,,ordu/fatsa\nmustafa kemal🇹🇷,sıla hasret🌵,slahasret9,1275534893532041216,Tue Jun 23 21:03:22 +0000 2020,"[{'source': 'description', 'pcode': 52}, {'sou...","[{'source': 'name', 'gender': 'female'}]","[1256584354391867392, 1277888419394445312, 122...","[1287350271186743297, 1276210699828690944, 125...","[{'twt_id_str': '1277550300577050624', 'twt_tx...",6,14,https://pbs.twimg.com/profile_images/127553502...,220526,active,https://twitter.com/slahasret9
2,"antalya, türkiye",tamam işte,ferit,feriteri,1344922664,Thu Apr 11 17:48:48 +0000 2013,"[{'source': 'location', 'pcode': 7}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1048911191144321028, 1270157658700632066, 138...","[1211058201736572928, 1448589090883096579, 810...","[{'twt_id_str': '1527347068620382213', 'twt_tx...",209,178,https://pbs.twimg.com/profile_images/152406047...,220604,active,https://twitter.com/feriteri
3,mersin anamur,eğlenmek,mustafamustafa,mustafa92675333,1322814149349486593,Sun Nov 01 08:14:22 +0000 2020,"[{'source': 'location', 'pcode': 33}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1413934914228084739, 1518170300441141249, 145...","[1259801988042297344, 1461001763356610564, 259...","[{'twt_id_str': '1523932231944871941', 'twt_tx...",8,163,https://pbs.twimg.com/profile_images/132281447...,220614,active,https://twitter.com/mustafa92675333
4,,istanbul\n1996\nmuscle,ozand,ozand06932981,1397148361954840578,Tue May 25 11:11:50 +0000 2021,"[{'source': 'description', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","[1388263839045668865, 291028871, 3240621950, 8...","[1388263839045668865, 1347647995869802496, 137...","[{'twt_id_str': '1412669844084908033', 'twt_tx...",4,25,https://pbs.twimg.com/profile_images/139715311...,220505,active,https://twitter.com/ozand06932981
5,,"“mutlu olmayı yarına bırakmak, karşıya geçmek ...",ardaa 💸,aardaa00,1376089807219200003,Sun Mar 28 08:32:50 +0000 2021,"[{'source': 'description', 'pcode': 33}]","[{'source': 'name', 'gender': 'male'}]","[1281616361001754626, 1250797435779264512, 200...","[1250797435779264512, 1281616361001754626, 130...","[{'twt_id_str': '1377679883321630723', 'twt_tx...",3,12,https://pbs.twimg.com/profile_images/137609021...,220507,active,https://twitter.com/aardaa00
6,"istanbul, türkiye",😒🤷‍♀️😭😂,melekturan,melektu36980983,1388968674988539905,Sun May 02 21:28:47 +0000 2021,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}, {'sou...","[631642410, 1189571058229268480, 7615859556608...","[743844888, 793847674495729664, 13885524419693...","[{'ref_twt_id_str': '1033489710767919105', 're...",4,26,https://pbs.twimg.com/profile_images/138896900...,220609,active,https://twitter.com/melektu36980983
7,beşiktaş jk 🖤,sa bahar ama siz baar deyin,bahar,baharkanbay,3654179601,Mon Sep 14 15:19:35 +0000 2015,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1269308207811432448, 1532008969002176514, 149...","[1269308207811432448, 1310265884359958528, 304...","[{'ref_twt_id_str': '1537943039426981890', 're...",1250,468,https://pbs.twimg.com/profile_images/153591283...,220618,active,https://twitter.com/baharkanbay
8,"istanbul, türkiye",social media management💻🗞,batuhan çavaş,batuhancavas,944149951,Mon Nov 12 17:38:31 +0000 2012,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gend

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
60,"avcılar, istanbul",instagram: harunaygunn,harun.,harunaygunn,1111383964382572544,Thu Mar 28 21:46:00 +0000 2019,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1533171306832834560, 1430835721556926464, 100...","[1533171306832834560, 1005963259109937157, 855...","[{'twt_id_str': '1537113224121835526', 'twt_tx...",300,405,https://pbs.twimg.com/profile_images/152434357...,220620,active,https://twitter.com/harunaygunn
61,"istanbul, türkiye",atü📚📚 elektirik\n\nfenerbahçe 💛💙💛💙,ishak çiftçi,isakpsa,1290522924504551424,Tue Aug 04 05:40:39 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","[1492108894411399174, 1424842159539343364, 272...","[924540200499412992, 1497950596611063815, 1492...","[{'twt_id_str': '1515363804724535307', 'twt_tx...",58,120,https://pbs.twimg.com/profile_images/152001809...,220527,active,https://twitter.com/isakpsa
62,bartın,yaşayan ölülerden,baran özkaynak,baran_ozkynk,1333689400727891969,Tue Dec 01 08:29:21 +0000 2020,"[{'source': 'location', 'pcode': 74}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1514676862647218179, 1038487293902942209, 148...","[3225952257, 1514676862647218179, 148130689104...","[{'twt_id_str': '1527338681744314372', 'twt_tx...",222,212,https://pbs.twimg.com/profile_images/146001849...,220524,active,https://twitter.com/baran_ozkynk
63,"ıstanbul, turkey",ıstanbul aydın university,fevzi,fevkaladefevz,1154420778768752640,Thu Jul 25 15:58:56 +0000 2019,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'name', 'gender': 'male'}]","[1366659251419111427, 1409425830455951362, 150...","[1366659251419111427, 1409425830455951362, 117...","[{'twt_id_str': '1529229871251132416', 'twt_tx...",179,64,https://pbs.twimg.com/profile_images/149405916...,220605,active,https://twitter.com/fevkaladefevz
64,istanbul,istanbul🌜 kartal,harun akyıldız,harun34563,1322545421630230530,Sat Oct 31 14:26:28 +0000 2020,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1325054473228722178, 1325130762803310592, 133...","[1336363881636225029, 1069253018397417472, 136...","[{'twt_id_str': '1362682438724554752', 'twt_tx...",60,159,https://pbs.twimg.com/profile_images/136155425...,220502,active,https://twitter.com/harun34563
65,"mardin merkez, mardin",genç fenerbahçeliler |5399|,ferhat admış,ferhatadmisgfb,958103295654092801,Mon Jan 29 22:23:21 +0000 2018,"[{'source': 'location', 'pcode': 47}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1450863268684341253, 1336715746542542849, 127...","[1494247005325082624, 1425871138530467840, 145...","[{'twt_id_str': '1463628367488602119', 'twt_tx...",63,68,https://pbs.twimg.com/profile_images/144120147...,220531,active,https://twitter.com/ferhatadmisgfb
66,"istanbul, türkiye",att 😷,hasan ile pideye doğru,hasancamuur,1053987544168308736,Sun Oct 21 12:33:07 +0000 2018,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1358385152397877250, 1440992475276095489, 103...","[1116963212556210176, 1038740144906481664, 144...","[{'ref_twt_id_str': '1534477585396092929', 're...",652,477,https://pbs.twimg.com/profile_images/152196664...,220609,active,https://twitter.com/hasancamuur
67,"eskisehir, turkey",🌹 🚬\n https://t.co/yccnstkysk,mustafa etike,etikemustafa,1214212027922563077,Mon Jan 06 15:48:22 +0000 2020,"[{'source': 'location', 'pcode': 26}]","[{'source': 'name', 'gender': 'male'}]","[1313171300060483584, 1151153768165076992, 127...","[1399355731573448710, 1307813369995501570, 113...","[{'twt_id_str': '1304097250075971586', 'twt_tx...",58,59,https://pbs.twimg.com/profile_images/130471589...,220424,active,https://twitter.com/etikemustafa
68,"van, türkiye",aşçı \nvan başkale \ninstagram: i

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
120,manisa,"kitap kurdu, amatör fotoğrafçı, kıllanan adam,...",mustafa selim daldal,edepya_hu,281539750,Wed Apr 13 13:44:44 +0000 2011,"[{'source': 'location', 'pcode': 45}]","[{'source': 'name', 'gender': 'male'}]","[328041541, 697429640441425923, 11842640959750...","[1275396815345631233, 3367372143, 704232814179...","[{'ref_twt_id_str': '1267172882708398089', 're...",46,57,https://pbs.twimg.com/profile_images/362294183...,220531,active,https://twitter.com/edepya_hu
121,"gaziantep, türkiye",sevdiklerimin kokusu değişmiş toprak kokar olm...,can yener,canynr0,1417412955038273538,Tue Jul 20 09:16:04 +0000 2021,"[{'source': 'location', 'pcode': 27}]","[{'source': 'name', 'gender': 'male'}]","[1274042581786275841, 1303118700573319169, 135...","[1169896585888616449, 1054694469843537923]","[{'twt_id_str': '1462191659903602688', 'twt_tx...",2,23,https://pbs.twimg.com/profile_images/146218986...,220620,active,https://twitter.com/canynr0
122,"bahawalpur, pakistan","educator, student of ınternational relations, ...",tahir faiz,maliktahirfaiz1,911868466902261760,Sun Sep 24 08:22:39 +0000 2017,"[{'source': 'description', 'pcode': 61}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1341939904574590976, 1502317131089784836, 128...","[3271192722, 2777061114, 1505429215302369280, ...","[{'ref_twt_id_str': '1505835678453575685', 're...",532,1370,https://pbs.twimg.com/profile_images/124365189...,220321,active,https://twitter.com/maliktahirfaiz1
123,,beden eğitimi öğretmeni 🤸🏼‍♂️ şahinbey beledi...,sevgi şahin,sevgiah90356674,1204101889580777472,Mon Dec 09 18:14:30 +0000 2019,"[{'source': 'description', 'pcode': 27}]","[{'source': 'name', 'gender': 'female'}, {'sou...","[1265695938737319936, 1259971340607590401, 125...","[1187340748452220930, 1197385509473017856, 126...","[{'twt_id_str': '1316081433132371973', 'twt_tx...",24,40,https://pbs.twimg.com/profile_images/120410235...,220516,active,https://twitter.com/sevgiah90356674
124,,allah'ın dediği olur..,şerafettin,erafett42760695,1191045102476693507,Sun Nov 03 17:31:24 +0000 2019,"[{'source': 'description', 'pcode': 25}]","[{'source': 'name', 'gender': 'male'}]","[1263843989226172416, 1278394051839111168, 133...","[1263843989226172416, 1380561269795778563, 135...","[{'ref_twt_id_str': '1360623998481145861', 're...",15,36,https://pbs.twimg.com/profile_images/134323821...,220516,active,https://twitter.com/erafett42760695
125,"diyarbakır, türkiye",güneşi yükseltenlere bin selam ..,yakup özcan,jacobs0021,1235186100110536704,Wed Mar 04 12:51:47 +0000 2020,"[{'source': 'location', 'pcode': 21}]","[{'source': 'name', 'gender': 'male'}]","[1059948651647287296, 3179621437, 126001578112...","[1430563852635230211, 1398363466797404165, 139...","[{'twt_id_str': '1419071854187360263', 'twt_tx...",13,13,https://pbs.twimg.com/profile_images/141798292...,220531,active,https://twitter.com/jacobs0021
126,"antalya, türkiye",antalya,selahattin çoban,selahattin_cb,1245180761830981633,Wed Apr 01 02:46:52 +0000 2020,"[{'source': 'location', 'pcode': 7}, {'source'...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[121096624, 1070413561468502016, 4736304015, 3...","[1261749189962326023, 1259725021989273601, 125...","[{'twt_id_str': '1246241885762383873', 'twt_tx...",17,63,https://pbs.twimg.com/profile_images/143830740...,220516,active,https://twitter.com/selahattin_cb
127,,beşiktaş’lı,d£m,mahsunn211,905819163859484673,Thu Sep 07 15:44:52 +0000 2017,"[{'source': 'description', 'pcode': 34}]","[{'source': 'screen_name', 'gender': 'male'}]","[1050155699085463554, 342562305, 230417407, 14...","[1050155699085463554, 1400379727358173190, 138...","[{'twt_id_str': '1487519529047625736', 'twt_tx...",11,32,https://pbs.twimg.com/profile_images/145697444...,220503,active,https://twitter.com/mahsunn211
128,"kahramanmaraş, türkiye",kürkçü dükkanı,ısmail,ısm

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
180,"istanbul, türkiye",hamide eylul💕👩‍👧☄,hamide eylul,hamideharun1,945406719294885890,Mon Dec 25 21:31:41 +0000 2017,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1063334397166977024, 832559464683335680, 7880...","[1440747428370472964, 1063334397166977024, 133...","[{'twt_id_str': '1330484908695953408', 'twt_tx...",15,57,https://pbs.twimg.com/profile_images/132329832...,220607,active,https://twitter.com/hamideharun1
181,"siirt, türkiye",istanbul üniversitesi,merve aydın,aydnmrv0,1069214306611412992,Sun Dec 02 12:58:50 +0000 2018,"[{'source': 'location', 'pcode': 56}, {'source...","[{'source': 'name', 'gender': 'female'}]","[1223995644152635393, 1524700817315573760, 145...","[1223995644152635393, 1457654564400869376, 150...","[{'ref_twt_id_str': '1523334330340175872', 're...",62,66,https://pbs.twimg.com/profile_images/151224878...,220530,active,https://twitter.com/aydnmrv0
182,,düşünüyorum!... şu koca atmosferde ben neyim? ...,senanur bilgin,bilginnsenanur,1408801121993465860,Sat Jun 26 14:55:55 +0000 2021,"[{'source': 'description', 'pcode': 25}]","[{'source': 'name', 'gender': 'female'}]","[1372539037114904577, 1054667082388910080, 127...","[1424583788097585156, 1420788155955953666, 141...","[{'twt_id_str': '1434111766259240967', 'twt_tx...",13,28,https://pbs.twimg.com/profile_images/143241741...,220519,active,https://twitter.com/bilginnsenanur
183,"istanbul, türkiye",bozarmı sandın acılar,sara mara 👽,ssaraydnn,1255742081437519872,Thu Apr 30 06:14:03 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}]","[1405274855084331011, 1250291059529093120, 148...","[1528827450918051842, 2602248268, 141776982327...","[{'twt_id_str': '1517574162486865922', 'twt_tx...",6536,6359,https://pbs.twimg.com/profile_images/141797342...,220524,active,https://twitter.com/ssaraydnn
184,"istanbul, türkiye","“hayat sana portakal veriyorsa, mermelat yap.”",gizem,gizemerlrr,1240769011517964289,Thu Mar 19 22:36:08 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}, {'sou...","[804999113611874304, 2484406933, 1366073870969...","[1366073870969618437, 2484406933, 104692764562...","[{'ref_twt_id_str': '1422446729496989697', 're...",8,24,https://pbs.twimg.com/profile_images/139773324...,220529,active,https://twitter.com/gizemerlrr
185,,sakarya,umutcan çırakoğlu,umutcanrakoglu,1233422816902549506,Fri Feb 28 16:05:06 +0000 2020,"[{'source': 'description', 'pcode': 54}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1100829516476358657, 936290933762609155, 1396...","[1377331888839401472, 1455576702491500548, 106...","[{'twt_id_str': '1461372725797310466', 'twt_tx...",4,9,https://pbs.twimg.com/profile_images/123342310...,220530,active,https://twitter.com/umutcanrakoglu
186,"çorlu, türkiye",fat gamer boi. don't try to add me if u r not ...,berkay küçükdeveci,berkaykucukd,843854463878225921,Mon Mar 20 15:59:17 +0000 2017,"[{'source': 'location', 'pcode': 59}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[2164603921, 486882044, 4221891466, 571206183,...","[1200376469337325568, 1039105289897877505]","[{'ref_twt_id_str': '1286315512440524800', 're...",2,6,https://pbs.twimg.com/profile_images/127078935...,220623,active,https://twitter.com/berkaykucukd
187,"yalova, türkiye",yalova,ibrahim,ibrhmmylmaz77,1363198997205237767,Sat Feb 20 18:49:18 +0000 2021,"[{'source': 'location', 'pcode': 77}, {'source...","[{'source': 'name', 'gender': 'male'}]","[1254373488409882624, 1333530692282699778, 981...","[1267554638800314370, 1283356375486271492, 125...","[{'twt_id_str': '1409499122550378501', 'twt_tx...",42,59,https://pbs.twimg.com/profile_images/136319917...,220604,active,https://twitter.com/ibrhmmylmaz77
188,"ataşehir, istanbul",𝑀𝑜𝑜𝓃𝓁𝒾𝑔𝒽𝓉,buğra dönmez,donmezbugraa,4245

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
240,turkiye elazığ,türkçe ağzımda anne sütü gibidir..\nbaskil'de ...,eray çolak,eraycolak3,879679521183719425,Tue Jun 27 12:35:16 +0000 2017,"[{'source': 'location', 'pcode': 23}, {'source...","[{'source': 'name', 'gender': 'male'}]","[1084458403051831302, 792130813, 1481634899731...","[1084458403051831302, 792130813, 1481634899731...","[{'ref_twt_id_str': '1531280222183739400', 're...",159,159,https://pbs.twimg.com/profile_images/125509389...,220613,active,https://twitter.com/eraycolak3
241,,mersin 🇹🇷,kübra nur,kubinur,1251270923946209280,Fri Apr 17 22:07:05 +0000 2020,"[{'source': 'description', 'pcode': 33}]","[{'source': 'name', 'gender': 'female'}]","[18971997, 1343242377629151233, 68034431, 1016...","[1486950997339557889, 411053359, 1054126079596...","[{'twt_id_str': '1371575429816053771', 'twt_tx...",4,6,https://pbs.twimg.com/profile_images/137157576...,220519,active,https://twitter.com/kubinur
242,"amasya, türkiye",❤️💙\n@trabzonspor,ahmetcan sarıkaya,ahmetcsarikaya,4851290176,Tue Jan 26 21:47:06 +0000 2016,"[{'source': 'location', 'pcode': 5}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1573354014, 315782848, 1227137403837722624, 1...","[1243968691533996032, 1447971279429160962, 145...","[{'twt_id_str': '1525808790821650434', 'twt_tx...",250,438,https://pbs.twimg.com/profile_images/150058144...,220515,active,https://twitter.com/ahmetcsarikaya
243,"erzincan, türkiye",@fenerbahce,taha değil talhaaa,muhammedtalhaı3,1245105365810909186,Tue Mar 31 21:47:17 +0000 2020,"[{'source': 'location', 'pcode': 24}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1405785967395590144, 1364850708512710658, 139...","[1476631947384344576, 1431349619737436163, 140...","[{'twt_id_str': '1361723481818300419', 'twt_tx...",52,56,https://pbs.twimg.com/profile_images/141964941...,220530,active,https://twitter.com/muhammedtalhaı3
244,,ak parti şehitkamil ilçe yönetim kurulu üyesi,ahmet kaya,ahmet_kaya2763,1316773646321549312,Thu Oct 15 16:11:30 +0000 2020,"[{'source': 'description', 'pcode': 27}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1002134495158513665, 216106605, 3126704435, 2...","[1452330144241233924, 1509243113235267598, 126...","[{'twt_id_str': '1488912622032502792', 'twt_tx...",112,89,https://pbs.twimg.com/profile_images/131677440...,220518,active,https://twitter.com/ahmet_kaya2763
245,"orhangazi, türkiye",beyazın kaderi kirlenmek siyahın kaderi suçlan...,çiçero,tevfik_durkut,1160133069648683009,Sat Aug 10 10:17:32 +0000 2019,"[{'source': 'location', 'pcode': 16}]","[{'source': 'screen_name', 'gender': 'male'}]","[2600210567, 533085085, 1315911842359844865, 1...","[1443604124986118145, 1315911842359844865, 129...","[{'twt_id_str': '1356623303532830721', 'twt_tx...",34,85,https://pbs.twimg.com/profile_images/116013324...,220429,active,https://twitter.com/tevfik_durkut
246,"kocaeli, türkiye",insanları kazanmak için kendinizi kaybetmeyin 🖤,azraaaa🖤,azzzzzrra,882922348772552704,Thu Jul 06 11:21:06 +0000 2017,"[{'source': 'location', 'pcode': 41}]","[{'source': 'name', 'gender': 'female'}]","[1445137624721674243, 2865612725, 2310561325, ...","[1070746236351905792, 392003868, 1220419840365...","[{'twt_id_str': '1532365635199672321', 'twt_tx...",128,66,https://pbs.twimg.com/profile_images/152923762...,220609,active,https://twitter.com/azzzzzrra
247,,istanbul esenyurt üniversitesi spor yöneticili...,reyhan,ryhankorkmaz,1186578360669032448,Tue Oct 22 09:41:56 +0000 2019,"[{'source': 'description', 'pcode': 34}, {'sou...","[{'source': 'name', 'gender': 'female'}]","[1258513981565698049, 1455107609778868225, 121...","[1258513981565698049, 1455107609778868225, 164...","[{'twt_id_str': '1532769011359965185', 'twt_tx...",20,25,https://pbs.twimg.com/profile_images/143312112...,220617,active,https://twitter.com/ryhankorkmaz
248,,fatih,fatih,fatihcakmkk_,1340

...

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
720,manisa/akhisar,milliyetci_beyy,yakup,yakup88185596,1296590988006359043,Thu Aug 20 23:32:51 +0000 2020,"[{'source': 'location', 'pcode': 45}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[240747852, 1209544733942591490, 217222331, 80...",[1209544733942591490],"[{'ref_twt_id_str': '1384580869537599500', 're...",1,13,https://pbs.twimg.com/profile_images/138479776...,220604,active,https://twitter.com/yakup88185596
721,"izmir, türkiye",sen nasıl bir aşksın fenerbahçe!!!,abdullah,herteldenyazan1,1286567029097603073,Fri Jul 24 07:41:08 +0000 2020,"[{'source': 'location', 'pcode': 35}]","[{'source': 'name', 'gender': 'male'}]","[37322526, 11525172, 1523646564647981058, 1494...","[1166931561893289984, 581938674, 1095231320211...","[{'twt_id_str': '1531699437407543300', 'twt_tx...",4,29,https://pbs.twimg.com/profile_images/143909941...,220601,active,https://twitter.com/herteldenyazan1
722,"ankara, türkiye","engineer, dog lover",semih kurttepeli 🇹🇷,semihkurttepeli,574396418,Tue May 08 11:22:24 +0000 2012,"[{'source': 'location', 'pcode': 6}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[190234372, 873297080281944064, 41446422, 9008...","[947952954744688640, 1142757509507231744, 1248...","[{'twt_id_str': '1519676697687306240', 'twt_tx...",1406,69,https://pbs.twimg.com/profile_images/103564604...,220510,active,https://twitter.com/semihkurttepeli
723,mersin,semillas fito a.ş.,hilmi ateş,hlmate1,1882524960,Thu Sep 19 09:35:29 +0000 2013,"[{'source': 'location', 'pcode': 33}]","[{'source': 'name', 'gender': 'male'}]","[871145500497649665, 1112331488035594240, 8778...","[74977562, 1366406284321435648, 417788742, 135...","[{'ref_twt_id_str': '1405469046368358405', 're...",28,39,https://pbs.twimg.com/profile_images/133026651...,220516,active,https://twitter.com/hlmate1
724,,adana,ibrahim samet,brahims96377073,1350483741811073026,Sat Jan 16 16:43:22 +0000 2021,"[{'source': 'description', 'pcode': 1}]","[{'source': 'name', 'gender': 'male'}]","[111688862, 469709673, 700131104, 129727316686...",[1350479108896120838],"[{'twt_id_str': '1350897753463906305', 'twt_tx...",1,6,https://pbs.twimg.com/profile_images/135048391...,220515,active,https://twitter.com/brahims96377073
725,istanbul,https://t.co/j0kyczqaeh\nhttps://t.co/uspc4sn0dc,fırat yaşa,firatyasaa,1156843311640039425,Thu Aug 01 08:25:13 +0000 2019,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1146294432, 2309770364, 504190705, 126571977]","[1146294432, 2309770364, 1183318237578223616]","[{'twt_id_str': '1504978451698229251', 'twt_tx...",3,4,https://pbs.twimg.com/profile_images/119057308...,220528,active,https://twitter.com/firatyasaa
726,"bursa, türkiye",fenerbahçeli 💛💙,emirhan,emirhan40787822,1350499425244880896,Sat Jan 16 17:45:41 +0000 2021,"[{'source': 'location', 'pcode': 16}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1455102562496024577, 1085508672527187968, 189...","[1455102562496024577, 946121866506391552, 1338...","[{'ref_twt_id_str': '1441386268533923840', 're...",6,11,https://pbs.twimg.com/profile_images/140690290...,220516,active,https://twitter.com/emirhan40787822
727,"istanbul, türkiye",hepsi manyak bunların’,kübra 🌑,kubrakadagann,1302015631081037824,Fri Sep 04 22:48:25 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}, {'sou...","[2998758401, 2966081218, 1512156281334870024, ...","[3064011190, 1908087380, 1046515875602419712, ...","[{'ref_twt_id_str': '1538600822837239810', 're...",79,113,https://pbs.twimg.com/profile_images/150308697...,220622,active,https://twitter.com/kubrakadagann
728,,🌿balıkesir //\nkonya- neü.,sabihaertrk,sabihaertrk10,1000837521633398790,Sun May 27 20:33:54 +0000 2018,"[{'source': 'description', 'pcode': 10}, {'sou...","[{'source': 'name', 'gender': 'female'}

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
780,,yahyalı kayseri🇹🇷,mustafa deniz,mustafa43293227,1114912788835721217,Sun Apr 07 15:28:17 +0000 2019,"[{'source': 'description', 'pcode': 38}, {'sou...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1201516732910837760, 1016198241417822208, 258...","[1062972731837333504, 94521775]","[{'twt_id_str': '1284822975443804161', 'twt_tx...",2,13,https://pbs.twimg.com/profile_images/111491336...,220505,active,https://twitter.com/mustafa43293227
781,"istanbul, türkiye",paramedik 🚑\nscirocco turkey,seda karagül,sedaakaragul,1256344840708796427,Fri May 01 22:08:55 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}]","[1259109669991256065, 1236386588168462336, 125...","[1253060740325486592, 2803423481, 125541497412...","[{'twt_id_str': '1493977731985518592', 'twt_tx...",151,254,https://pbs.twimg.com/profile_images/148643335...,220423,active,https://twitter.com/sedaakaragul
782,eskişehir,marmara üniversitesi ‘,zehra güney,zehraaguney,369674077,Wed Sep 07 19:07:58 +0000 2011,"[{'source': 'location', 'pcode': 26}, {'source...","[{'source': 'name', 'gender': 'female'}, {'sou...","[2246262702, 3722631381, 1016198241417822208, ...","[1276922208984666113, 1068570540447801345, 940...","[{'ref_twt_id_str': '1111985665720418307', 're...",175,5,https://pbs.twimg.com/profile_images/140157454...,220425,active,https://twitter.com/zehraaguney
783,batman/ankara,gülhane eczacılık 4/5💊,seloşş,selinbulutt0,1138243575551057920,Tue Jun 11 00:36:30 +0000 2019,"[{'source': 'location', 'pcode': 6}, {'source'...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1243159200315977730, 1507315684925128711, 150...","[1524754120208527360, 1507315684925128711, 150...","[{'twt_id_str': '1485901802079989760', 'twt_tx...",113,139,https://pbs.twimg.com/profile_images/137407075...,220602,active,https://twitter.com/selinbulutt0
784,"istanbul, türkiye",ve dostun biri sana kötülük ederse..,nazlı karaca,nzlkrcaa,1025437709190529024,Fri Aug 03 17:46:16 +0000 2018,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}]","[212885668, 1953691460, 1193475391484309506, 1...","[2370153240, 330494713, 1358830389582852101, 1...","[{'ref_twt_id_str': '1420670544840646656', 're...",234,284,https://pbs.twimg.com/profile_images/124016134...,220430,active,https://twitter.com/nzlkrcaa
785,"erzincan, türkiye",erzincan/tercan/çadırkaya,kaan,kaan99028974,1297879847629398017,Mon Aug 24 12:54:17 +0000 2020,"[{'source': 'location', 'pcode': 24}, {'source...","[{'source': 'name', 'gender': 'male'}]","[1298738829311041536, 1218932577219891200, 119...","[1359988997225934851, 1305599569842114562, 121...","[{'twt_id_str': '1298884999949758465', 'twt_tx...",7,19,https://pbs.twimg.com/profile_images/129788826...,220519,active,https://twitter.com/kaan99028974
786,"avcılar, istanbul",seni yaşatacak neler var neler @fenerbahce,muratcan,mrtcnkpln18,1220394085569220611,Thu Jan 23 17:13:32 +0000 2020,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'name', 'gender': 'male'}]","[1134215474521264128, 1292895005460312066, 143...","[1134215474521264128, 1435820442158309376, 149...","[{'ref_twt_id_str': '1517924888090234880', 're...",209,210,https://pbs.twimg.com/profile_images/151417600...,220424,active,https://twitter.com/mrtcnkpln18
787,"adana, türkiye",ne ara bizi istemeyen insanları özleyecek kada...,betül_knt,kntbetul,1338166793744109572,Sun Dec 13 17:00:17 +0000 2020,"[{'source': 'location', 'pcode': 1}]","[{'source': 'name', 'gender': 'female'}]","[1421807030310932482, 1489694897519173642, 149...","[1488443265754013697, 1491871208819810310, 138...","[{'twt_id_str': '1513602236315750417', 'twt_tx...",123,115,https://pbs.twimg.com/profile_images/151034057...,220511,active,https://twitter.com/kntbetul
788,,elifsu \n^ ankara,elifsu,elifsu34601121,1283735

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
840,"ankara, türkiye",pilot çırağı gariban,yusuf turan,turan820,2208932177,Thu Dec 05 15:35:07 +0000 2013,"[{'source': 'location', 'pcode': 6}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1210254370941677569, 2300920058, 132474801943...","[2300920058, 1324748019431018498, 121025437094...","[{'twt_id_str': '1419930104482959360', 'twt_tx...",42,35,https://pbs.twimg.com/profile_images/133749556...,220519,active,https://twitter.com/turan820
841,,istanbul üniversitesi sosyal hizmet,dilan,dilan95066523,1340200642955137024,Sat Dec 19 07:42:09 +0000 2020,"[{'source': 'description', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}, {'sou...","[1343710200, 3537145515, 429847858, 1293281622...",[1293281622184296448],"[{'ref_twt_id_str': '1342402090501623808', 're...",1,5,https://abs.twimg.com/sticky/default_profile_i...,220615,active,https://twitter.com/dilan95066523
842,"ıstanbul, turkey",air traffic controller,göktuğ kuş,kusgoktug,1228361837214883840,Fri Feb 14 16:54:33 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","[1378202977, 276799677, 1445484963097235473, 1...","[2519741653, 3021966886, 1238909858767757312, ...","[{'ref_twt_id_str': '1448188612873490432', 're...",8,60,https://pbs.twimg.com/profile_images/122836240...,220429,active,https://twitter.com/kusgoktug
843,"adana, türkiye",elifsu,osmanutlu,osmanutlu,712813803990155264,Thu Mar 24 01:30:29 +0000 2016,"[{'source': 'location', 'pcode': 1}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[3382834149, 1234037945121853440, 115123840840...","[994944606189088768, 186035135]","[{'ref_twt_id_str': '1524108769512374272', 're...",2,76,https://pbs.twimg.com/profile_images/126393811...,220515,active,https://twitter.com/osmanutlu
844,"istanbul, türkiye",hayatımız kaydı,buse,minyonlarice,1266430799521427457,Fri May 29 18:06:57 +0000 2020,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}]","[4468722269, 1378059186764931079, 144680029978...","[1511038954795159555, 1378059186764931079, 144...","[{'twt_id_str': '1520435144255016962', 'twt_tx...",48,39,https://pbs.twimg.com/profile_images/150480949...,220430,active,https://twitter.com/minyonlarice
845,"istanbul, türkiye",sade yaşam,bilal,bilal03121912,1202658252674256896,Thu Dec 05 18:37:53 +0000 2019,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[348772117, 2978328988, 1102199180, 438741755,...",[34146256],"[{'twt_id_str': '1453099939131691020', 'twt_tx...",1,40,https://pbs.twimg.com/profile_images/126748312...,220501,active,https://twitter.com/bilal03121912
846,,eskişehir,muhammet ensar #freeporcay,muhamme87533026,1356720013953089546,Tue Feb 02 21:44:07 +0000 2021,"[{'source': 'description', 'pcode': 26}]","[{'source': 'name', 'gender': 'male'}]","[1275492238458576896, 1148601001869553664, 800...","[1231313359653494786, 1309859791058219010, 136...","[{'ref_twt_id_str': '1388453636150280199', 're...",81,156,https://pbs.twimg.com/profile_images/138602575...,220614,active,https://twitter.com/muhamme87533026
847,"sinop, türkiye",sinop durağan ak parti kadın kolları başkanı,nazan akkaya,nazanakkaya57,1425923157609811969,Thu Aug 12 20:52:45 +0000 2021,"[{'source': 'location', 'pcode': 57}, {'source...","[{'source': 'name', 'gender': 'female'}, {'sou...","[1041179787501469697, 1433684705317113873, 153...","[984906520344358912, 1077679545669640192, 1433...","[{'ref_twt_id_str': '1536093717932740610', 're...",49,133,https://pbs.twimg.com/profile_images/142876418...,220613,active,https://twitter.com/nazanakkaya57
848,"artvin, türkiye",zümraaa,özge,zge63028513,1092899422244098049,Tue Feb 05 21:35:02 +0000 2019,"[{'source': 'location', 'pcode': 8}]","[{'source': 'name', 'gender': 'female'}]","[163848722, 1012618229200670720, 1016198241417...",[75

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
900,,gaziantep\nsusmak en büyük sanattır.,kemal yılırım,kemalylrm2,1222233182055038977,Tue Jan 28 19:01:20 +0000 2020,"[{'source': 'description', 'pcode': 27}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[128332009, 1370267989, 1093445462856814592, 1...",[1366401285990330368],"[{'twt_id_str': '1222521116830380032', 'twt_tx...",1,24,https://pbs.twimg.com/profile_images/122223447...,220615,active,https://twitter.com/kemalylrm2
901,istanbul,#istanbulsözleşmesiyaşatır,baharflex,baharbutflex,1095053538600861696,Mon Feb 11 20:14:43 +0000 2019,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1112420930251354113, 1261976504013111296, 192...","[1112420930251354113, 797118640235708416, 9924...","[{'twt_id_str': '1388920030386065415', 'twt_tx...",5,18,https://pbs.twimg.com/profile_images/138020893...,220507,active,https://twitter.com/baharbutflex
902,"bayrampaşa, istanbul",galatasaray,atakan özbek,atakanozbekkk,1296388860,Sun Mar 24 19:42:52 +0000 2013,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[69008563, 1499876576225374219, 357086475, 148...","[1460919252156919814, 1499876576225374219, 148...","[{'twt_id_str': '1500175212075225090', 'twt_tx...",43,163,https://pbs.twimg.com/profile_images/150005470...,220427,active,https://twitter.com/atakanozbekkk
903,"adiyaman, turkey",besyo 1/4 📚⚽,ömer faruk güngör⚡,omergungorr5,1218179072599240704,Fri Jan 17 14:31:57 +0000 2020,"[{'source': 'location', 'pcode': 2}]","[{'source': 'name', 'gender': 'male'}]","[1311970480602189825, 1513857156055027714, 145...","[1495756228143767553, 1466115560639504393, 124...","[{'twt_id_str': '1515815886061510665', 'twt_tx...",141,186,https://pbs.twimg.com/profile_images/151090840...,220423,active,https://twitter.com/omergungorr5
904,"diyarbakır, türkiye",iyi insanlar.!\n sağlıkçı,r🌻,kartalicexd,1381845875954216966,Tue Apr 13 05:45:13 +0000 2021,"[{'source': 'location', 'pcode': 21}]","[{'source': 'screen_name', 'gender': 'female'}]","[1482785667281309702, 1266416223048343552, 128...","[1083721191435718656, 1301416790136938497, 125...","[{'twt_id_str': '1502563281063530499', 'twt_tx...",265,286,https://pbs.twimg.com/profile_images/150037808...,220513,active,https://twitter.com/kartalicexd
905,karaman 🤝🤝,yıkmayı bırak sallayamassiniz bile,hasan avcı,hasanav83507874,1362791761970683907,Fri Feb 19 15:51:01 +0000 2021,"[{'source': 'location', 'pcode': 70}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[1362839020930289664, 1235200075468505088, 576...","[301081079, 1362839020930289664, 1350430875436...","[{'twt_id_str': '1363461581204574209', 'twt_tx...",3,16,https://pbs.twimg.com/profile_images/136279468...,220528,active,https://twitter.com/hasanav83507874
906,"adana, türkiye",yakın doğu üniversitesi/paramedik,emir,emirkzlhn,1147829818827911168,Sun Jul 07 11:28:49 +0000 2019,"[{'source': 'location', 'pcode': 1}]","[{'source': 'name', 'gender': 'male'}]","[928319087754739713, 114794628, 294744259, 441...","[868576305201205250, 950413293390303232, 36881...","[{'ref_twt_id_str': '1496559107704303617', 're...",42,55,https://pbs.twimg.com/profile_images/139565556...,220617,active,https://twitter.com/emirkzlhn
907,"aydın, türkiye",adü nurseee,zeynep,revirablaa,830081519385444353,Fri Feb 10 15:50:31 +0000 2017,"[{'source': 'location', 'pcode': 9}]","[{'source': 'name', 'gender': 'female'}]","[1488957238089814022, 1140940658875805696, 306...","[1140940658875805696, 1488957238089814022, 143...","[{'twt_id_str': '1485549858371031042', 'twt_tx...",584,179,https://pbs.twimg.com/profile_images/137447795...,220512,active,https://twitter.com/revirablaa
908,istanbul,hayatıma mutluluklar dilerim,ceren,mmaybebabe,845326844383649792,Fri Mar 24 17:29:59 +0000 2017,"[{'source': 'location', 'pcode': 34}]","

,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,followers,tweets,followers_count,following_count,pp,downloaded,activity,url
960,hatay/dörtyol,hatay/dörtyol sana aşığım çünkü imkansızsın,yaşar can bozkurt,yaarcanbozkurt1,1430592644233809920,Wed Aug 25 18:08:00 +0000 2021,"[{'source': 'location', 'pcode': 31}, {'source...","[{'source': 'name', 'gender': 'male'}]","[153531196, 2826514966, 241994168, 2533403804,...","[1521948861161295872, 153531196, 302581509]","[{'ref_twt_id_str': '1524053708442677248', 're...",3,40,https://pbs.twimg.com/profile_images/143059349...,220519,active,https://twitter.com/yaarcanbozkurt1
961,"izmir, türkiye",bla bla,ege arca,egearca08,1293054826176274434,Tue Aug 11 05:21:20 +0000 2020,"[{'source': 'location', 'pcode': 35}]","[{'source': 'name', 'gender': 'male'}]","[708141166152302592, 2924688458, 1145825695941...",[708141166152302592],"[{'twt_id_str': '1293548608692912129', 'twt_tx...",1,11,https://pbs.twimg.com/profile_images/129354908...,220616,active,https://twitter.com/egearca08
962,"sivas merkez, sivas",togü|pdr,melike ipek,mekossaa,1437477044850511881,Mon Sep 13 18:03:42 +0000 2021,"[{'source': 'location', 'pcode': 58}, {'source...","[{'source': 'name', 'gender': 'male'}]","[1518867687056855041, 1304874189619429378, 150...","[1518867687056855041, 1304874189619429378, 368...","[{'ref_twt_id_str': '1502388044938285059', 're...",120,182,https://pbs.twimg.com/profile_images/148789394...,220529,active,https://twitter.com/mekossaa
963,"afyonkarahisar, türkiye",sıradan bir insan,polat,canakman0,1321959851686301696,Thu Oct 29 23:39:37 +0000 2020,"[{'source': 'location', 'pcode': 3}]","[{'source': 'name', 'gender': 'male'}]","[550869506, 1238877653173784577, 1356288078923...","[1096715944871821314, 1248995854033264651, 550...","[{'twt_id_str': '1450563673521524743', 'twt_tx...",15,26,https://pbs.twimg.com/profile_images/139659223...,220523,active,https://twitter.com/canakman0
964,,"proud pakistani,follower of great ımran khan| ...",ihsan khan,ihsanmcitp,752847423601508353,Tue Jul 12 12:49:47 +0000 2016,"[{'source': 'description', 'pcode': 61}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[4088885357, 1367395458881122307, 115701616, 1...","[810470362433683457, 846619680555065344, 61943...","[{'ref_twt_id_str': '1394327616824487936', 're...",1879,4347,https://pbs.twimg.com/profile_images/123451931...,220524,active,https://twitter.com/ihsanmcitp
965,"istanbul, türkiye",military waste & law veteran & art worker (ask...,ikrami özturan,mrmunificent,2305865620,Sun Jan 26 15:15:15 +0000 2014,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'female'}]","[725961054388408320, 581214167, 14282536164822...","[2435709176, 1451782422, 562665002, 1493643422...","[{'ref_twt_id_str': '1509583964989759503', 're...",780,129,https://pbs.twimg.com/profile_images/134782857...,220426,active,https://twitter.com/mrmunificent
966,,malatya \n14.05.2006,kerem doğan,keremdo67988600,1425799199656923146,Thu Aug 12 12:40:15 +0000 2021,"[{'source': 'description', 'pcode': 44}]","[{'source': 'name', 'gender': 'male'}, {'sourc...","[4829799209, 23186079, 18426124]",[],"[{'ref_twt_id_str': '1467841532598403074', 're...",0,3,https://pbs.twimg.com/profile_images/142579929...,220613,active,https://twitter.com/keremdo67988600
967,"sancaktepe, istanbul",_#kars. li...!!!,fatih taşöz,fatihtaz1,1262594633319813121,Tue May 19 04:03:29 +0000 2020,"[{'source': 'location', 'pcode': 34}, {'source...","[{'source': 'name', 'gender': 'male'}, {'sourc...","[23186079, 1256219613593047042, 89462113490957...","[3873331953, 1083233898136514560, 873670807444...","[{'twt_id_str': '1263041513665241088', 'twt_tx...",8,24,https://pbs.twimg.com/profile_images/126259507...,220527,active,https://twitter.com/fatihtaz1
968,"hatay, türkiye","duralım burada,güzel esiyor..",halil ibrahim özer,halilib05203356,1304713144586174465,Sat Sep 12 09:27:20 +0000 2020,"[{'source': 'location', 'pcode': 31}]","[{'s

In [28]:
sample_df.columns

Index(['location', 'description', 'name', 'screen_name', 'id_str',
       'created_at', 'province_codes', 'genders', 'following', 'followers',
       'tweets', 'followers_count', 'following_count', 'pp', 'downloaded',
       'activity', 'url'],
      dtype='object')

In [30]:
sample_df = sample_df[['url', 'location', 'description', 'name', 'screen_name', 'id_str',
                       'created_at', 'province_codes', 'genders', 'following', 'followers',
                       'tweets', 'followers_count', 'following_count', 'pp', 'downloaded']]

In [33]:
sample_df.to_excel("thousand_sample_users-220702.xlsx", index=False)